<h1 align=center><font size = 8>Capstone Final Project - The Battle of Neighborhoods</font></h1>

# A. Business Problem: Introduction

#### Our client has several business modules up and running in Manhattan, Newyork. He wants to start a new business in Brooklyn, Newyork. The client is unware of booming businesses in Brooklyn and wants to start with something which is not available in any particular area. 
#### We need to suggest a few areas and businesses for his new business in Brooklyn. 

# B. Data Requirement and Collection

#### Collect Newyork data from IBM and filter Brooklyn data using FourSquare API. The data will be initially downloaded in JSON file. 
#### Convert JSON data into pandas dataframe. Extract the required details such as Neighborhood, Categories, Longitude, Latitude etc. for Brooklyn. 


#### 1. Download/import the required libraries

In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py

<a id='item1'></a>


#### 2. Download and Explore Dataset

Download Newyork data from json file using wget command. Load and explore json data. Print the first item in the list

In [46]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

#### 3. Transform the data into pandas dataframe

In [47]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

neighborhoods.head()

The dataframe has 5 boroughs and 306 neighborhoods.


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### 4. Get the latitude and longitude values of New York City. Create a map of New York with neighborhoods superimposed on top.

In [4]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### 5. Get Brooklyn data from Newyork data

In [48]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


#### 6. Get geographical coordinates of Brooklyn. Create visualization of Brooklyn Neighborhood 

In [6]:
address = 'Brooklyn, NY'

geolocator = Nominatim(user_agent="br_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


#### 7. Define Foursquare Credentials and Version
Utilize the Foursquare API to explore the neighborhoods and segment them


In [69]:
CLIENT_ID = '' 
CLIENT_SECRET = '' 
ACCESS_TOKEN = ''
VERSION = '20180604'
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### 8. Let's explore the first neighborhood in our Brooklyn dataframe


In [50]:
neighborhood_latitude = brooklyn_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = brooklyn_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = brooklyn_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


#### 9. Extract the top 100 venues that are in Bay Ridge within a radius of 1000 meters
Create GET request URL. Store json data in results variable


In [51]:
# type your answer here
radius=1000
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

results = requests.get(url).json()

#### 10. Define get_category_type function to extract the category of venues from the Foursquare lab


In [52]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### 11. Clean json data and structure it into pandas dataframe


In [53]:
venues =results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues returned by Foursquare:'.format(nearby_venues.shape[0]))

nearby_venues.head()

92 venues returned by Foursquare:


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Exquisite Delight,Caribbean Restaurant,40.649091,-73.949243
1,Jus Juice,Juice Bar,40.652208,-73.949723
2,"Courts (Furniture, Electronics, & Appliances)",Furniture / Home Store,40.650436,-73.950608
3,Tastee Pattee,Caribbean Restaurant,40.650850,-73.947838
4,Little Roy,Coffee Shop,40.651437,-73.949671


#### 12. Explore Neighborhoods in Brooklyn
Define a function to repeat the same process to all the neighborhoods in Brooklyn

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### 13. Call the above function on each neighborhood and create a new dataframe called brooklyn_venues


In [55]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

print(brooklyn_venues.shape)
print(brooklyn_venues.head())
#brooklyn_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus
(5626, 7)
  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Bay Ridge              40.625801          

# C. Methodology

#### Explore each Neighborhood and extract top common venues. Sort the venues in descending order and display the top 10 venues for each neighborhood to understand the data more.
#### Apply **KMEANS CLUSTERING** to partition Brooklyn neighborhoods data in different clusters. Once clustering is done, we can analyze which cluster has better scopes for our client in Brooklyn.

#### 14. Analyze Each Neighborhood


In [56]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

print(brooklyn_onehot.shape)

brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
print(brooklyn_grouped.shape)
brooklyn_grouped.head()

(5626, 351)
(70, 351)


,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Baggage Locker,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cruise Ship,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Kofte Place,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Piercing Parlor,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Post Office,Pub,Racetrack,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roller Rink,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Sh

#### 15. Extract each neighborhood along with the top 5 most common venues

In [57]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0           Pizza Place  0.08
1   Japanese Restaurant  0.05
2    Chinese Restaurant  0.05
3                  Bank  0.05
4  Cantonese Restaurant  0.04


----Bay Ridge----
                venue  freq
0         Pizza Place  0.08
1                 Spa  0.07
2  Italian Restaurant  0.05
3    Greek Restaurant  0.03
4      Sandwich Place  0.03


----Bedford Stuyvesant----
         venue  freq
0  Coffee Shop  0.12
1          Bar  0.06
2         Café  0.06
3  Pizza Place  0.05
4    Wine Shop  0.04


----Bensonhurst----
                venue  freq
0                Bank  0.08
1              Bakery  0.05
2         Pizza Place  0.05
3  Italian Restaurant  0.05
4  Chinese Restaurant  0.04


----Bergen Beach----
                 venue  freq
0          Supermarket   0.1
1         Liquor Store   0.1
2        Deli / Bodega   0.1
3  American Restaurant   0.1
4   Italian Restaurant   0.1


----Boerum Hill----
           venue  freq
0            Bar  0.07
1  

#### 16. Define a function to sort the venues in descending order

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### 17. Create a new dataframe and display the top 10 venues for each neighborhood

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Pizza Place,Bank,Japanese Restaurant,Chinese Restaurant,Bakery,Cantonese Restaurant,Sushi Restaurant,Bubble Tea Shop,Italian Restaurant,Coffee Shop
1,Bay Ridge,Pizza Place,Spa,Italian Restaurant,American Restaurant,Middle Eastern Restaurant,Bar,Sandwich Place,Bagel Shop,Greek Restaurant,Diner
2,Bedford Stuyvesant,Coffee Shop,Bar,Café,Pizza Place,Wine Shop,Deli / Bodega,Cocktail Bar,Juice Bar,Mexican Restaurant,Discount Store
3,Bensonhurst,Bank,Italian Restaurant,Bakery,Pizza Place,Japanese Restaurant,Chinese Restaurant,Mobile Phone Shop,Cantonese Restaurant,Sandwich Place,Bagel Shop
4,Bergen Beach,Italian Restaurant,Donut Shop,Stables,Baseball Field,Peruvian Restaurant,Liquor Store,Pizza Place,Supermarket,Deli / Bodega,American Restaurant


#### 18. Cluster Neighborhoods using KMeans

In [60]:
# set number of clusters
kclusters = 5

brooklyn_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 1, 4, 2, 1, 1, 0, 2], dtype=int32)

#### 19. Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [61]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,2,Pizza Place,Spa,Italian Restaurant,American Restaurant,Middle Eastern Restaurant,Bar,Sandwich Place,Bagel Shop,Greek Restaurant,Diner
1,Brooklyn,Bensonhurst,40.611009,-73.995180,1,Bank,Italian Restaurant,Bakery,Pizza Place,Japanese Restaurant,Chinese Restaurant,Mobile Phone Shop,Cantonese Restaurant,Sandwich Place,Bagel Shop
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Bakery,Pizza Place,Mexican Restaurant,Bank,Chinese Restaurant,Latin American Restaurant,Ice Cream Shop,Sandwich Place,Tea Room,Szechuan Restaurant
3,Brooklyn,Greenpoint,40.730201,-73.954241,2,Bar,Coffee Shop,Cocktail Bar,Pizza Place,Bakery,Café,Record Shop,French Restaurant,Polish Restaurant,Yoga Studio
4,Brooklyn,Gravesend,40.595260,-73.973471,1,Pizza Place,Bar,Donut Shop,Bakery,Chinese Restaurant,Grocery Store,Lounge,Bank,Plaza,Bagel Shop


# D. Results
#### Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Analyze all the final clusters and find out the best cluster and business for the client. 

#### 20. Examine Clusters and results

#### Cluster 1


In [62]:
cluster1= brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]
cluster1

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Flatbush,Caribbean Restaurant,Pizza Place,Coffee Shop,Mexican Restaurant,Bar,Restaurant,Middle Eastern Restaurant,Café,Discount Store,Mobile Phone Shop
10,East Flatbush,Caribbean Restaurant,Discount Store,Supermarket,Grocery Store,Liquor Store,Bakery,Lounge,Gym,Performing Arts Venue,Check Cashing Service
14,Brownsville,Discount Store,Pizza Place,Supermarket,Mobile Phone Shop,Pharmacy,Fried Chicken Joint,Liquor Store,Sandwich Place,Caribbean Restaurant,Gas Station
25,Cypress Hills,Fast Food Restaurant,Latin American Restaurant,Grocery Store,Sandwich Place,Pizza Place,Donut Shop,Chinese Restaurant,Discount Store,Ice Cream Shop,Food
26,East New York,Chinese Restaurant,Pizza Place,Supermarket,Pharmacy,Yoga Studio,Basketball Court,Metro Station,Spanish Restaurant,Park,Caribbean Restaurant
28,Canarsie,Caribbean Restaurant,Pizza Place,Bank,Pharmacy,Grocery Store,Video Game Store,Food,Taco Place,Mobile Phone Shop,Fast Food Restaurant
29,Flatlands,Caribbean Restaurant,Chinese Restaurant,Supermarket,Discount Store,Pharmacy,Fried Chicken Joint,Donut Shop,Grocery Store,Lounge,Park
42,Prospect Lefferts Gardens,Caribbean Restaurant,Pizza Place,Café,Bakery,Wine Shop,Pharmacy,Liquor Store,Cocktail Bar,Coffee Shop,Italian Restaurant
43,Ocean Hill,Discount Store,Fried Chicken Joint,Café,Fast Food Restaurant,Southern / Soul Food Restaurant,Donut Shop,Pharmacy,Mexican Restaurant,Deli / Bodega,Supermarket
44,City Line,Pizza Place,Fast Food Restaurant,Pharmacy,Grocery Store,Chinese Restaurant,Supermarket,Donut Shop,Fried Chicken Joint,Food,South American Restaurant


#### Cluster 2


In [63]:
cluster2= brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]
cluster2

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bensonhurst,Bank,Italian Restaurant,Bakery,Pizza Place,Japanese Restaurant,Chinese Restaurant,Mobile Phone Shop,Cantonese Restaurant,Sandwich Place,Bagel Shop
2,Sunset Park,Bakery,Pizza Place,Mexican Restaurant,Bank,Chinese Restaurant,Latin American Restaurant,Ice Cream Shop,Sandwich Place,Tea Room,Szechuan Restaurant
4,Gravesend,Pizza Place,Bar,Donut Shop,Bakery,Chinese Restaurant,Grocery Store,Lounge,Bank,Plaza,Bagel Shop
5,Brighton Beach,Beach,Bakery,Restaurant,Eastern European Restaurant,Seafood Restaurant,Pharmacy,Aquarium,Café,Russian Restaurant,Food Truck
6,Sheepshead Bay,Italian Restaurant,Sushi Restaurant,Russian Restaurant,Café,Turkish Restaurant,Seafood Restaurant,Bank,Sandwich Place,Pizza Place,Dessert Shop
7,Manhattan Terrace,Pizza Place,Bank,Donut Shop,Bagel Shop,Mobile Phone Shop,Grocery Store,Sushi Restaurant,Pharmacy,Convenience Store,Japanese Restaurant
11,Kensington,Pizza Place,Café,Grocery Store,Gas Station,Thai Restaurant,Bakery,Spa,Ice Cream Shop,Mexican Restaurant,Deli / Bodega
27,Starrett City,Pizza Place,Department Store,Mobile Phone Shop,Pharmacy,Furniture / Home Store,Park,Bus Stop,Clothing Store,Discount Store,Chinese Restaurant
32,Coney Island,Theme Park Ride / Attraction,Theme Park,Pizza Place,Pharmacy,Hot Dog Joint,Beach,Deli / Bodega,Coffee Shop,Bar,Arcade
33,Bath Beach,Pizza Place,Bank,Japanese Restaurant,Chinese Restaurant,Bakery,Cantonese Restaurant,Sushi Restaurant,Bubble Tea Shop,Italian Restaurant,Coffee Shop


#### Cluster 3


In [64]:
cluster3= brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]
cluster3

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Pizza Place,Spa,Italian Restaurant,American Restaurant,Middle Eastern Restaurant,Bar,Sandwich Place,Bagel Shop,Greek Restaurant,Diner
3,Greenpoint,Bar,Coffee Shop,Cocktail Bar,Pizza Place,Bakery,Café,Record Shop,French Restaurant,Polish Restaurant,Yoga Studio
9,Crown Heights,Café,Caribbean Restaurant,Southern / Soul Food Restaurant,Grocery Store,Pizza Place,Bar,Coffee Shop,Japanese Restaurant,Deli / Bodega,Bakery
12,Windsor Terrace,Italian Restaurant,Café,Deli / Bodega,Coffee Shop,Playground,Wine Shop,Diner,Middle Eastern Restaurant,Plaza,American Restaurant
13,Prospect Heights,Bar,Wine Shop,Cocktail Bar,Ice Cream Shop,Restaurant,Thai Restaurant,Mexican Restaurant,Sushi Restaurant,New American Restaurant,Coffee Shop
15,Williamsburg,Pizza Place,Bar,Wine Bar,Coffee Shop,American Restaurant,Yoga Studio,Bakery,Breakfast Spot,Salon / Barbershop,Liquor Store
16,Bushwick,Bar,Coffee Shop,Mexican Restaurant,Pizza Place,Cocktail Bar,Grocery Store,Deli / Bodega,Italian Restaurant,Record Shop,Music Venue
17,Bedford Stuyvesant,Coffee Shop,Bar,Café,Pizza Place,Wine Shop,Deli / Bodega,Cocktail Bar,Juice Bar,Mexican Restaurant,Discount Store
18,Brooklyn Heights,Park,Italian Restaurant,Pizza Place,Wine Shop,Coffee Shop,Yoga Studio,Grocery Store,Pier,Bakery,Japanese Restaurant
19,Cobble Hill,Italian Restaurant,Cocktail Bar,Pizza Place,Bakery,Bar,Yoga Studio,Wine Shop,Cosmetics Shop,Grocery Store,Bagel Shop


#### Cluster 4


In [65]:
cluster4= brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]
cluster4

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Sea Gate,Beach,Supermarket,Pharmacy,Donut Shop,Spa,Chinese Restaurant,Pizza Place,Park,Fish Market,Food & Drink Shop


#### Cluster 5


In [66]:
cluster5= brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]
cluster5

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Bergen Beach,Italian Restaurant,Donut Shop,Stables,Baseball Field,Peruvian Restaurant,Liquor Store,Pizza Place,Supermarket,Deli / Bodega,American Restaurant


# E. Conclusion
#### There are no Italian restuarants in cluster 1 Neighborhoods in top 10 venues. Therefore, we can suggest our client to open an Italian restaurant in the neighborhoods cluster 1. 
#### Similarily, there are no Caribbean Restaurant in cluster 2 Neighborhoods in top 10 venues. The client can also open a Caribbean Restaurant in the neighborhoods of cluster 2. 

In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**5 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters